Generate private data

In [2]:
import cv2
import os
import random
import torch
import mediapipe as mp
import tqdm
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils


In [ ]:
#cv2 read video
cap = cv2.VideoCapture('private_dataset/Xiang.mp4')

def generate_frame(path, frame_number):
    #create folder if it doesn't exist
    if not os.path.exists('Private_dataset/' + path):
        os.makedirs('Private_dataset/' + path)
        

    vidcap = cv2.VideoCapture('Private_dataset/{}.mp4'.format(path))
    #sample 40 frames uniformly from the video
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = sorted(random.sample(range(frame_count), frame_number))
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_indices[0])
    success, image = vidcap.read()
    count = 0
    face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
    res = []
    print(frame_indices)
    
    for i in range(1, frame_count):
        
        if i == frame_indices[count]:
            #rotate the image 180 degrees
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.detections:
                for detection in results.detections:
                    #get the bounding box
                    bbox = detection.location_data.relative_bounding_box
                    #get the center of the bounding box
                    center_x = bbox.xmin + bbox.width / 2
                    center_y = bbox.ymin + bbox.height / 2 

                    center_x = int(center_x * image.shape[1])
                    center_y = int(center_y * image.shape[0])

                    size = int(max(bbox.width * image.shape[1], bbox.height * image.shape[0])*1)
                    size = int(size/2) + 10
                    image = image[center_y-size-5:center_y+size-5, center_x-size:center_x+size]
                    try:
                        #rotate the image 180 degrees
                        image = cv2.rotate(image, cv2.ROTATE_180)
                        cv2.imwrite(f"./Private_dataset/{format(path)}/{format(count, '04d')}.jpg", image)
                        print(f"./Private_dataset/{format(path)}/{format(count, '04d')}.jpg")
                        res.append(image)
                    except:
                        print("error")
            else:
                print("no face detected")

            count += 1
            if count >= len(frame_indices):
                    break
        success, image = vidcap.read()
        #print(success)
        if not success:
            break

generate_frame('Xiang', 40)


### Store generated data

In [7]:
import os
from random import sample

src_path = './CelebA_HQ_facial_identity_dataset/train/'
des_path = './Dataset/train/'

# loop through all the subfolders in the dataset
for subfolder in os.listdir(src_path):
    subfolder_path = os.path.join(src_path, subfolder)

    # check if the subfolder contains more than 20 images
    if len(os.listdir(subfolder_path)) > 20:
        # sample 20 images uniformly from the subfolder
        image_filenames = sample(os.listdir(subfolder_path), 20)

        # copy the sampled images to the destination folder
        for image_filename in image_filenames:
            image_path = os.path.join(subfolder_path, image_filename)
            os.system('cp {} {}'.format(image_path, des_path))
            

In [4]:
subfolder_path

'./CelebA_HQ_facial_identity_dataset/train/Xiang/0002.jpg'